In [1]:
import model_tools as mt
import pandas as pd
import numpy as np
import requests
import secret
import time
import datetime

In [2]:
def alpha_data(SYMBOLS: list = None, DATASET: str = None, save_path: str = "Data/data.csv"):
    secrets = secret.data["alpha_vantage"]

    if SYMBOLS is None:
        SYMBOLS = pd.read_csv("sp500stocks.csv")["Symbol"].tolist()[:5]

    # Define your API parameters in a dictionary
    if DATASET is None:
        DATASET = secrets["default_dataset"]

    symbol_data = []
    for DATA in DATASET["other"].keys():
        params = {'function': DATA, 'apikey': secrets["api_key"]}
        params.update(DATASET["other"][DATA])
        response = requests.get(secrets["base_url"], params=params)
        data = response.json()
        df = mt.parse_response_data(data)
        symbol_data.append(df)

    econ_df = pd.concat(symbol_data, axis=1, join='outer')

    combined_data = []

    for SYMBOL in sorted(SYMBOLS):
        symbol_data = []

        for DATA in DATASET["symbol_req"].keys():
            start_time = datetime.datetime.now() 
            print(DATA, " for ", SYMBOL)

            # Construct API call with specific parameters
            symbol_params = {'function': DATA, 'symbol': SYMBOL, 'apikey': secrets["api_key"]}
            symbol_params.update(DATASET["symbol_req"][DATA])
            
            # Make request
            response = requests.get(secrets["base_url"], params=symbol_params)
            data = response.json()
            
            # Parse to dataframe
            df = mt.parse_response_data(data)
            # Check if df has any columns that already exist in symbol_data
            # If so, merge the data instead of appending df as a new dataframe
            if symbol_data:
                for col in df.columns:
                    if col in symbol_data[0].columns:
                        # Merge the column data
                        symbol_data[0][col] = symbol_data[0][col].combine_first(df[col])
                    else:
                        # If column is not in symbol_data, add it
                        symbol_data[0][col] = df[col]
            else:
                symbol_data.append(df)

            elapsed_time = (datetime.datetime.now() - start_time).total_seconds() 
            sleep_time = max(0, 60 / secrets["rate_limit"] - elapsed_time) 
            time.sleep(sleep_time)

        # Combine data for this symbol across all datasets
        if symbol_data:
            symbol_data.append(econ_df)
            symbol_combined = pd.concat(symbol_data, axis=1, join='outer')
            # Instead of inserting Ticker one by one, collect them and add later
            combined_data.append(symbol_combined.assign(Ticker=SYMBOL))

    # Combine data across all symbols
    if combined_data:
        final_combined_data = pd.concat(combined_data, axis=0, join='outer')
        final_combined_data['Datetime'] = pd.to_datetime(final_combined_data.index)
        final_combined_data.reset_index(drop=True, inplace=True)

        # Ensure 'Datetime' and 'ticker' are the first two columns
        cols = final_combined_data.columns.tolist()
        cols.insert(0, cols.pop(cols.index('Ticker')))  # Move 'Ticker' to the first column
        cols.insert(1, cols.pop(cols.index('Datetime')))  # Move 'Datetime' to the second column
        final_combined_data = final_combined_data[cols]

        # Save to CSV
        final_combined_data.to_csv(save_path)
        return final_combined_data

data = alpha_data(SYMBOLS=["AAPL", "MSFT", "TSLA", "NVDA", "SNPS", "HPE", "STNE", "CLVT", "CRM", "SAP", "GOOG", "AMD", "INTC", "AVGO", "ORCL", "TSM", "CSCO", "SEDG", "TTD", "PYPL", "BABA",], DATASET=secret.data["alpha_vantage"]["default_dataset"])

data["Datetime"] = pd.to_datetime(data["Datetime"])

# "AAPL", "MSFT", "TSLA", "NVDA", "SNPS", "HPE", "STNE", "CLVT", "CRM", "SAP", "GOOG",
# Selecting only the data with 'Datetime' between 2018-12-31 and 2023-12-31
data = data[pd.Timestamp('2023-12-31') > data['Datetime']]
data = data[pd.Timestamp('2018-12-31') < data['Datetime']]

columns_to_drop = ["date", "value", "fiscalDateEnding",]

settings = mt.load_settings()
# Drop columns only if they exist in the DataFrame
for col in columns_to_drop:
    if col in data.columns:
        data.drop(columns=col, inplace=True)

data.ffill(limit=91, inplace=True)
data.bfill(limit=91, inplace=True)
data = data.T.drop_duplicates().T
data.drop_duplicates(ignore_index=True, inplace=True)
data.replace(["None", None, "", np.nan, pd.NA], [0, 0, 0, 0, 0], inplace=True)

data = mt.optimize_data_types(data)
print(data)
print(data.info(verbose=True))
data.to_csv("Data/data.csv")
data, settings = mt.preprocess(data, settings)
print(data)
print(data.info(verbose=True))

TIME_SERIES_DAILY_ADJUSTED  for  AAPL
INCOME_STATEMENT  for  AAPL
BALANCE_SHEET  for  AAPL
CASH_FLOW  for  AAPL
EARNINGS  for  AAPL
TIME_SERIES_DAILY_ADJUSTED  for  AMD
INCOME_STATEMENT  for  AMD
BALANCE_SHEET  for  AMD
CASH_FLOW  for  AMD
EARNINGS  for  AMD
TIME_SERIES_DAILY_ADJUSTED  for  AVGO
INCOME_STATEMENT  for  AVGO
BALANCE_SHEET  for  AVGO
CASH_FLOW  for  AVGO
EARNINGS  for  AVGO
TIME_SERIES_DAILY_ADJUSTED  for  BABA
INCOME_STATEMENT  for  BABA
BALANCE_SHEET  for  BABA
CASH_FLOW  for  BABA
EARNINGS  for  BABA
TIME_SERIES_DAILY_ADJUSTED  for  CLVT
INCOME_STATEMENT  for  CLVT
BALANCE_SHEET  for  CLVT
CASH_FLOW  for  CLVT
EARNINGS  for  CLVT
TIME_SERIES_DAILY_ADJUSTED  for  CRM
INCOME_STATEMENT  for  CRM
BALANCE_SHEET  for  CRM
CASH_FLOW  for  CRM
EARNINGS  for  CRM
TIME_SERIES_DAILY_ADJUSTED  for  CSCO
INCOME_STATEMENT  for  CSCO
BALANCE_SHEET  for  CSCO
CASH_FLOW  for  CSCO
EARNINGS  for  CSCO
TIME_SERIES_DAILY_ADJUSTED  for  GOOG
INCOME_STATEMENT  for  GOOG
BALANCE_SHEET  for  

c:\Users\roman\Desktop\Bot\model_tools.py:447: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data[col] = pd.to_datetime(data[col], errors='coerce')


Dropped 96 columns due to low feature correlation with target
New data shape: (38325, 13)
       Ticker  Datetime      open      high       low     close  \
0           0         1 -0.200806 -0.196533 -0.195679 -0.192871   
1           0         2 -0.200806 -0.196533 -0.195679 -0.192871   
2           0         3 -0.230103 -0.231567 -0.229126 -0.234985   
3           0         4 -0.228760 -0.224243 -0.224365 -0.218750   
4           0         5 -0.228760 -0.224243 -0.224365 -0.218750   
...       ...       ...       ...       ...       ...       ...   
38320      20      1821 -0.419434 -0.422852 -0.419434 -0.419678   
38321      20      1822 -0.418945 -0.421143 -0.416260 -0.418457   
38322      20      1823 -0.419189 -0.421387 -0.416748 -0.420410   
38323      20      1824 -0.420654 -0.423828 -0.421143 -0.424072   
38324      20      1825 -0.420654 -0.423828 -0.421143 -0.424072   

       adjusted close  comprehensiveIncomeNetOfTax  currentNetReceivables  \
0           -0.748535       

In [3]:
df = pd.DataFrame([[2, None, None, 0],
                   [None, 4, None, 1],
                   [3, None, None, None],
                   [None, 3, None, 4]],
                  columns=list("ABCD"))

print(df)
df.ffill(inplace=True)
print(df)
# Example usage
base_path = 'Data'  # Replace with your base directory path
combined_dataset = mt.create_combined_timeseries_dataset(base_path)
print(combined_dataset)
combined_dataset.to_csv("new_data.csv")

     A    B     C    D
0  2.0  NaN  None  0.0
1  NaN  4.0  None  1.0
2  3.0  NaN  None  NaN
3  NaN  3.0  None  4.0
     A    B     C    D
0  2.0  NaN  None  0.0
1  2.0  4.0  None  1.0
2  3.0  4.0  None  1.0
3  3.0  3.0  None  4.0


ValueError: No objects to concatenate